# Parsing Syntax

We'll build a simple parsing library for our lisp based on Haskell's awesome [Parsec](https://wiki.haskell.org/Parsec) toolset. There's nothing novel here, so if you're willing to trust that these parsers work, feel free to skip ahead to [Building ASTs](#Building-ASTs). 

However, if you're not familiar with Parsec and are willing to stick around, I'll make the case in this section that parsing is a great entry point for people looking to learn about how functional programming can cleanly express complex operations involving handling failure without repeating yourself. 

The basic units we'll use to build our parsers are `item` and `unit`, defined as follows.

In [1]:
def item(stream):
    if not stream:
        return None, []
        
    return stream[0], stream[1:]

In [2]:
unit = lambda obj: lambda stream: (obj, stream)

Item is a sort of prototype for all of the parsers we'll build in this section, and also the workhorse of the parser apparatus. There are a couple of things to notice about the structure of `item`. 

Firstly, we've chosen to express parsers as a transformation of an input stream into an output object and a new input stream, representing the information that our parser has not consumed. 

`item("abc")` returns `"a", "bc"`, because it takes a single item from the input stream, and returns it along with the items it has not yet parsed.

The function is also capable of failure if it's given an empty input stream, and represents the inability to parse an output object with `None`.

Interestingly, `unit` has the same parser structure as `item`, but this time we construct a parser that returns the same output object regardless of the input stream, as well as the unmodified input stream. This has the effect of promoting any object to a parser. We'll find this a useful building block for stringing together a bunch of parsers, collecting their outputs, and then returning those outputs without consuming any additional input. A simple approach to this problem is outlined below.

Suppose we have inputs like these:

Well-formed input examples:

    numbers 123 (1)
    letters abc (2)
    
Malformed input examples:

    numbers abc (3)
    abc         (4)
    
We have a couple of goals when we parse:
 - check that our input is well-formed (lines 3 and 4 are malformed)
 - if the input is well-formed, extract useful information from it (in lines 1 and 2, for example, the "numbers" and "letters" tags are useful to us mainly to tell which parser we need to use, but need not be included in the output)
 
Let's also say, that we've already written parsers called `numbers` and `letters`, which parse a list of numbers or letters, respectively, and a parser called `numbers_or_letters`, which returns `True` if it consumes the string `"numbers "`, `False` if it consumes the string `"letters "`, and `None` if it is unable to consume either string.
 
Our immediate challenge is that stringing together these three parsers doesn't get us anywhere unless we can inspect the intermediate output of each parser. We need a function that takes a parser and a set of instructions for creating a second parser and fuses the first and second parsers into a third parser that represents the entire operation on the input stream. 

For our problem, this function call -- we'll call it `then` -- would look something like this:

    my_parser = then(numbers_or_letters, lambda is_number: numbers if is_number else letters)

Why is this way of constructing parsers special? For one thing, at no point in this entire expression have we needed to do error handling, or say anything about how the input streams should be allocated to each parser. We've abstracted away the plumbing of this complex function and kept only the aspects of the parser we want to talk about. 

Whether a string is malformed on the first or second parser makes no difference to us if we only want to know if the input stream as a whole is well-formed. And, if any parser fails, we don't have to worry about how much input intermediate parsers have consumed.

In [3]:
def then(p, g):
    def f(stream):
        parsed, rest = p(stream)
        if parsed is None:
            return None, stream
        
        parsed, rest = g(parsed)(rest)
        if parsed is None:
            return None, stream
        
        return parsed, rest
    
    return f

What about transforming an output once it's parsed? Is there a way to create a new parser that maps a function onto the result of the old parser?

In [4]:
pmap = lambda g, p: then(p, lambda parsed: unit(g(parsed)))

By the way, if you're interested in the fact that you can derive `pmap` from `then` and `unit`, check out [Functors and Monads](https://wiki.haskell.org/All_About_Monads#Introduction) for a cool discussion of how the patterns we've laid out on this page appear in an interesting variety of places. 

How about triggering failure manually without having to deal with the results? Can we use simple predicates to check whether an output object is valid?

In [5]:
fail = unit(None)

pred = lambda p: then(item, lambda x: unit(x) if p(x) else fail)

## Playing with Parsers

Make sure these examples make sense before you move on.

In [6]:
unit("test")('2')

('test', '2')

In [7]:
pred(lambda x: x == '2')(['2', '1'])

('2', ['1'])

In [8]:
pred(lambda x: x == '2')('21') 
# The parsers in this notebook don't care about the type of your input stream so long as it has list-like properties

('2', '1')

In [9]:
pred(lambda x: x == '2')('22')
# Pred parses only a single item from the input stream, 
# so it doesn't matter whether or not the rest of the stream is well-formed

('2', '2')

## Why Parser Combinators?

When people talk about building software, the analogy of building a structure from uniform blocks often arises. You might hear that good software is built from easily composable units -- pieces that fit together to form larger pieces, which fit together to form larger pieces, and so on, such that on any scale the novelty of a structure is expressed in the arrangement of the pieces, not their shapes. 

The thing is, we rarely have the luxury of dealing with smooth building blocks. Parsing is a great example of how even the smallest and most simple operations, like `item` or `unit` are a jagged mess to put together on their own. Building combinators, in my experience, is about gaining intuition about the kind of adapters we can attach to the complex wiring of our problems.

If you've made it here, you're through the hard part already. By building functions like `then` and `pmap`, we've sanded the edges off of the building blocks of the next section. Every parser we build from here to the end of this article will be a rearrangement of the primitive parsers that come before it.


# Parsing Strings
Up until now, we've worked with parsers general enough to operate on any sequence of objects as inputs. In the interest of time, we'll write the remainder of our parsers to work with sequences of characters.

In [10]:
char = lambda c: pred(lambda d: c == d)

In [11]:
stringify = lambda p: pmap(''.join, p)
listify   = lambda p: pmap(lambda x: [x], p)

In [12]:
def sequence(ps):
    if not ps:
        return unit([])
    
    p, ps = ps[0], ps[1:]
    return then(p, lambda parsed: then(sequence(ps), lambda parsed_list: unit([parsed] + parsed_list)))

In [13]:
string = lambda s: stringify(sequence([char(c) for c in s]))

## Playing with string parsers

In [14]:
then(char('c'), lambda x: char('d'))('cd') # unless we explicitly do something with x, it is discarded by then.

('d', '')

In [15]:
then(char('c'), lambda x: then(char('d'), lambda y: unit(x + y)))('cd')

('cd', '')

In [16]:
string("Hello")("Goodbye") # string parsers that fail consume no output, as expected

(None, 'Goodbye')

In [17]:
string("Hello")("Hello, World!") # string parsers consume only the input that they parse

('Hello', ', World!')

In [18]:
stream = "Hello, World!"
print(string("Hello")(stream))

stream # Additionally, string parsers are purely functional and do not modify the initial input stream

('Hello', ', World!')


'Hello, World!'

# Backtracking Combinators

What if we want to construct a parser that responds differently to failure? The `choice` function gives us the ability to pick which parser we wish to apply to the stream by selecting the first parser which can parse the input without failing.

In [19]:
def choice(ps):
    def f(stream):
        for p in ps:
            parsed, rest = p(stream)
            if parsed is not None:
                return parsed, rest
        
        return None, stream
    return f

In [20]:
optional = lambda p: choice([p, unit([])])

In [21]:
many = lambda p: then(p, lambda x: then(optional(many(p)), lambda l: unit([x] + l)))
some = lambda p: optional(many(p))

In other words, `many` of a pattern is just the pattern and `some` more of the pattern, and `some` of the pattern is optionally `many`. Pretty cool, eh?

In [22]:
between = lambda o, c, b: then(o, lambda _: then(b, lambda ret: then(c, lambda __: unit(ret))))

## Playing with Backtracking

In [23]:
some(string("cc"))("ccc")

(['cc'], 'c')

In [24]:
many(char("C"))("CCC")

(['C', 'C', 'C'], '')

In [25]:
many(string("CC"))("c")

(None, 'c')

In [26]:
some(string("CC"))("c")

([], 'c')

In [27]:
between(char('('), char(')'), many(char('c')))('(cccc)')

(['c', 'c', 'c', 'c'], '')

# Composing Parser Combinators

## Building sets to test inclusion
Let's define some sets that we can use for membership parsers

In [28]:
alpha_range = lambda a, b: [chr(n) for n in range(ord(a), ord(b) + 1)]

In [29]:
alphabet_set = set(alpha_range('a', 'z') + alpha_range('A', 'Z'))
digits_set = set(alpha_range('0', '9'))
whitespace_set = set([" ", "\n"])
symbols_set = set([c for c in "+-!@#$%^&*[]_;.,"])
starting_chars_set = alphabet_set.union(symbols_set)
body_chars_set = starting_chars_set.union(digits_set)

In [30]:
one_of = lambda s: pred(lambda x: x in s)
not_one_of = lambda s: pred(lambda x: x not in s)

## Parsers based on set membership

In [31]:
whitespace = stringify(many(one_of(whitespace_set)))
digit = one_of(digits_set)
alpha = one_of(alphabet_set)
alpha_string = stringify(many(alpha))
symbol = one_of(symbols_set)
starting_chars = many(one_of(starting_chars_set))
body_chars = some(one_of(body_chars_set))

## Stripping away uninteresting information 

In [32]:
some_snd = lambda d, p: some(then(d, lambda _: then(p, lambda y: unit(y))))
strip_ends = lambda d, p: pmap(lambda x: x[1], sequence([optional(d), p, optional(d)]))
optional_fst = lambda p, q: pmap(lambda x: x[0] + x[1], sequence([optional(listify(p)), q]))
sepby = lambda d, p: strip_ends(d, optional_fst(p, some_snd(d, p)))

## Parsing Numbers

This is the first big field test of our parser apparatus. We first define unsigned integers and build this up to a full-fledged scientific number parser.

In [33]:
uinteger = pmap(int, stringify(many(digit)))
udot_float = pmap(lambda x: float(f"{x[0]}.{x[2]}"), sequence([uinteger, char('.'), uinteger]))
signed = lambda n: then(optional(char('-')), lambda sign: then(n, lambda x: unit(x * (-1 if sign else 1))))
integer = signed(uinteger)
dot_float = signed(udot_float)
sci_float = pmap(lambda x: float(f"{x[0]}e{x[2]}"), sequence([choice([dot_float, integer]), char('e'), integer]))
number = choice([sci_float, dot_float, integer])

## Variable Names, String Literals, and Lists

This is specific to the syntax of our particular lisp implementation, but good practice here is that variables can start with symbols or letters but not numbers, and may contain numbers.

String literals are defined by anything surrounded by quotes, and lists are defined by anything surrounded by parens

In [34]:
code_word = stringify(then(starting_chars, lambda start: then(body_chars, lambda end: unit(start + end))))
double_quoted = stringify(between(char('"'), char('"'), some(not_one_of('"'))))
single_quoted = stringify(between(char("'"), char("'"), some(not_one_of("'"))))
quoted = choice([single_quoted, double_quoted])
parens = lambda p: between(char("("), char(")"), p)

## Playing With Composing Parsers

In [35]:
some(choice([char('c'), char('d')]))('dccdc')

(['d', 'c', 'c', 'd', 'c'], '')

In [36]:
many(integer)("123 456")

([123], ' 456')

In [37]:
sepby(whitespace, choice([number, code_word]))("123  456  789 ligma_123 'life is good'")

([123, 456, 789, 'ligma_123'], "'life is good'")

In [38]:
sepby(whitespace, alpha_string)("abc")

(['abc'], '')

In [39]:
sepby(whitespace, alpha_string)("abc")

(['abc'], '')

In [40]:
alpha_string("abc def")

('abc', ' def')

In [41]:
between(string('(('), string('))'), many(choice([code_word, whitespace])))("((abc def  ghi))")

(['abc', ' ', 'def', '  ', 'ghi'], '')

In [42]:
dot_float("123.345")

(123.345, '')

In [43]:
sci_float("-1.45e-2")

(-0.0145, '')

In [44]:
number("1.234e-1")

(0.1234, '')

In [45]:
number("12.23")

(12.23, '')

In [46]:
code_word("_alpha_123 - 2")

('_alpha_123', ' - 2')

In [47]:
quoted('"abc"')

('abc', '')

# Building ASTs

Now that we have the parsers we'll need for our syntax, we'll create classes to tag the syntax as we build our trees. This will help us distinguish a variable named `a` from the string literal `"a"`. 

In [48]:
class SyntaxElement:
    def __init__(self, value):
        self.value = value
    
    def __repr__(self):
        return f"{self.name} {self.value}"

In [49]:
class Literal(SyntaxElement):
    name = "Literal"

In [50]:
class Symbol(SyntaxElement):
    name = "Symbol"

Now, we can slap these labels onto the parsers we've created, and our abstract syntax tree parser is complete.

In [51]:
symbol = pmap(Symbol, code_word)

In [52]:
string_literal = pmap(Literal, quoted)

In [53]:
num_literal = pmap(Literal, number)

Note that operation and atom are mutually recursive because an operation contains many atoms, and an atom may be an operation. 

To help the Python interpreter sort this out, we'll modify our top-level choice combinator to accept suspended computations using lambdas to create ad hoc "thunks". More on this when we talk about our language's evaluation model.

In [54]:
operation = lambda atom: parens(sepby(whitespace, atom))

In [55]:
def lazy_choice(lps):
    def f(stream):
        for lp in lps:
            parsed, rest = lp()(stream)
            if parsed is not None:
                return parsed, rest
        
        return None, stream
    return f

In [56]:
atom = lazy_choice([ lambda: num_literal
                   , lambda: string_literal
                   , lambda: symbol
                   , lambda: operation(atom) ])

## Playing with Building ASTs

In [57]:
str(Literal(2))

'Literal 2'

In [58]:
Literal(2)

Literal 2

In [59]:
atom('(+ 1e-1 (a "b" c) ((+ 1 2) 3 4 5))')

([Symbol +,
  Literal 0.1,
  [Symbol a, Literal b, Symbol c],
  [[Symbol +, Literal 1, Literal 2], Literal 3, Literal 4, Literal 5]],
 '')

# So now what?

Now that we have a fully functioning lisp parser, we're ready to jump into evaluating our code! Not quite... 

Before our evaluator can start digesting our code, we have to cook it into something our runtime system can eat.

Exactly how much preparation we do in this "cooking" phase is determined by whether we want our language to be interpreted or compiled.

It all boils down (no pun intended) to exactly how picky of an eater our runtime system is. A CPU, for example, is the most discerning consumer of code, requiring the operations it executes to be spoon-fed to it -- forcing repetition and unreadability in exchange for simplicity of language components. Like an internal combustion engine, the fuel that is pumped into it must be refined almost beyond recognition, so that the parts it contains are so small and uniform that they can be processed by simple mechanical action.

A goat, on the other hand, is famous for eating indiscriminately, [caring little](https://zooatlanta.org/what-goats-really-eat/) whether its food comes from a trough, a lawn, or [the bark of a tree](https://www.youtube.com/watch?v=JGYUnaNIAZ4). This is because a goat carries within itself the machinery to break down a much wider palette of food into usable energy. Runtime evaluation systems that behave like goats need little refinement in the information they consume because they have built-in methods for breaking down complex syntax into usable instructions.

In the natural world, it's hard to draw a fine line between preparing a meal ends and digesting it. Some snakes, for example, begin digesting their prey before they've finished hunting them, by injecting powerful hemotoxins into their victims' bloodstream that tear the animal apart from the inside even as it tries to escape its hunter. Similar patterns emerge when we consider compilers that fully evaluate operations on literals known at compile-time, and metaprogramming constructs that unroll loops and inline function calls to create faster sequences of operations.

The point I'm making here is that there's rarely a clean distinction between preprocessing and evaluating code, and to decide whether a language is compiled or interpreted is to compare the relative complexities of the runtime environment and the interpreter. Behind every relatively simple car engine is a vast oil refinery apparatus, and behind the intricate digestive system of any goat, there need only be the most rudimentary decision procedure for what is edible.

What does this mean for our lisp? 

We want it to be a goat-viper chimera. In other words, the "compiler" for our code should produce a modified AST that is still human-readable but is desugared and optimized to the greatest extent possible. This leaves the door open for us to build a compiler for the "core" desugared language to target more low-level runtime environments.

# Preprocessing ASTs

The desugaring operations that we'll use in our preprocessor are these:
* `curry` turns lambdas of multiple arguments into nested lambda closures with a single argument. For example `(lambda (a b c) x)` becomes `(curried-lambda a (curried-lambda b (curried-lambda c x))))`.
* `associate` turns applications of multiple arguments to an operator like `(f a b c)` into nested single-argument applications like `(((f a) b) c)`. 
* `(set key value)` creates a hook that our evaluator will implement, which sets the runtime environment of `key` to `value`.
* `(defun (f a b c) x)` desugars to `(set f (lambda (a b c)  x))`
* `begin` creates a subenvironment and runs each of its arguments inside of it, returning the result of the last argument. For example, `(begin 1 (set a 2) a)` returns `2`, and does not affect the binding of `a` outside of the `begin`. 

In [60]:
def curry(arg_names, body):
    if not arg_names:
        return body
    
    return [Symbol("curried-lambda"), arg_names[0], curry(arg_names[1:], body)]

In [61]:
def associate(code):
    if not code[2:]:
        return [code[0], desugar(code[1])]
    
    operator = code[0]
    first_arg = desugar(code[1])
    return associate([[operator, first_arg]] + code[2:])

The `desugar` function figures out when to apply each syntactic sugar so that none of them infringe on the others. For example, arguments to `begin` should not be wrapped in left associated parentheses (`(((begin a) b) c)` is not a valid desugaring of `(begin a b c)`).

`desugar` also creates the runtime hooks that our evaluator will implement, `SetEnv`, `Procedure`, and `Begin`, as well as re-tagging syntax elements like `Symbol` and `Literal` to `Variable` and `Value`, respectively.

In [62]:
def desugar(expr):
    if isinstance(expr, list):
        if not expr:
            return expr
        
        operator, rest = desugar(expr[0]), expr[1:]
        if not rest:
            return [operator]
        
        if isinstance(operator, Variable):
            if operator.name == "curried-lambda":
                arg_name, body = expr[1].value, desugar(expr[2])
                return Procedure(arg_name, body)
            elif operator.name == "lambda":
                arg_names, body = expr[1], desugar(expr[2])
                return desugar(curry(arg_names, body))
            elif operator.name == "set":
                name, value = expr[1].value, desugar(expr[2])
                return SetEnv(name, value)
            elif operator.name == "defun":
                description, body = expr[1], desugar(expr[2])
                name, arg_names = description[0], description[1:]
                return desugar([Symbol("set"), name, [Symbol("lambda"), arg_names, body]])
            elif operator.name == "begin":
                subexprs = desugar([subexpr for subexpr in expr[1:]])
                return Begin(subexprs)
            
        return associate([desugar(operator)] + rest)
    
    elif isinstance(expr, Literal): 
        return Value(expr.value)
    elif isinstance(expr, Symbol):
        return Variable(expr.value)
        
    return expr

# Interpreting ASTs into Runtime Objects

We've already encountered `Variable`, `Value`, `Begin`, `SetEnv`, and `Procedure` as objects that we're feeding to our evaluator. In this section, we'll introduce `Interpreted` as an interface between these hooks and the default evaluation model.

These hooks are designed to be unopinionated about the evaluator interface as much as possible. This way, defining a new evaluator is as simple as creating a new interface and extending each of our runtime objects to implement it. 

Our default evaluation model will simply hook these objects into the python runtime using python's built-in lambdas and a [souped up dictionary class](#Representing-Environments) to represent the environment, but these objects are general enough to extend to other interpretation models.

In [63]:
class Interpreted:
    def interpret(self, env):
        raise NotImplementedError
        
    def replace(self, name, value):
        raise NotImplementedError

In [64]:
class Variable(Interpreted):
    def __init__(self, name):
        self.name = name
        
    def interpret(self, env):
        return env[self.name]
    
    def replace(self, name, value):
        if self.name == name:
            return value
        return self

class Value(Interpreted):
    def __init__(self, value):
        self.value = value
        
    def interpret(self, env):
        return self.value
    
    def replace(self, name, value):
        return self

class Begin(Interpreted):
    def __init__(self, exprs):
        self.exprs = exprs
        
    def interpret(self, env):
        subenv = env.push()
        result = None
        for subexpr in self.exprs:
            result = interpret(subexpr, subenv)
            
        return result
    
    def replace(self, name, value):
        return Begin(replace(self.exprs, name, value))

class SetEnv(Interpreted):
    def __init__(self, name, value):
        self.name = name
        self.value = value
    
    def interpret(self, env):
        env[self.name] = self.value
        return None
    
    def replace(self, name, value):
        return self

class Procedure(Interpreted):
    def __init__(self, arg_name, body):
        self.arg_name = arg_name
        self.body = body
    
    def interpret(self, env):
        return lambda arg: (replace(self.body, self.arg_name, arg))
    
    def replace(self, name, value):
        return Procedure(self.arg_name, replace(self.body, name, value))
    
    def __repr__(self):
        return f"(curried-lambda ({self.arg_name}) {self.body})"

In [65]:
def replace(expr, name, value):
    if isinstance(expr, RuntimeObject):
        return expr.replace(name, value)
    elif isinstance(expr, list):
        return [replace(subexpr, name, value) for subexpr in expr]

# Putting it all together

In [66]:
def evaluate(expr, env):
    if isinstance(expr, list):
        operator = evaluate(expr[0], env)
        if not expr[1:]:
            return operator
        if isinstance(operator, Procedure):
            operator = evaluate(operator, env)
        operand = expr[1]
        return evaluate(operator(operand), env)
    elif isinstance(expr, Interpreted):
        return evaluate(expr.interpret(env), env) 
        # Procedures returned by partial application must be evaluated again to become python lambdas.

    else:
        return expr

You may have noticed that `evaluate` only forces the evaluation of the operator, leaving the rest of the evaluation to happen when this first argument is applied to other (unevaluated) values. Why would this evaluation model help us?

I'll try answer this with a couple of examples:

Suppose we wish to evaluate the following code:


    (defun (true a b) a)
    (defun (false a b) b)
    (defun (cons a b f) (f a b))
    (defun (head l) (l true))
    (defun (tail l) (l false))
    (set ones (cons 1 ones))
    (head (tail ones))

    
What should the result of this code block be?
    
A naive interpreter might give up when it sees `ones` inside the set block, claiming that since `ones` was not defined on a prior line, it is meaningless. An interpreter that supports recursion might substitute the definition we have created into `ones`, creating an infinite process that bottoms out before we can extract even the first or second values from this infinite list.

We could also force the programmer to specify when they wish a value to be "lazy", or evaluated by need and not by default, by wrapping such values in lambda expressions where the evaluator cannot touch them until the lambda has been applied to an argument. Our code block under a "strict" system might work by modifying the last two lines like this:

    (set lazy-ones (cons 1 (lambda (arbitrary) ones)))
    (head ((tail ones) "arbitrary"))
    
The argument applied to `(tail ones)` is arbitrary, and only needed because our language doesn't support zero-argument lambdas. Laziness eliminates the need for such constructs.

Additionally, laziness enables us to avoid evaluating expressions that aren't needed in a final result. For example, consider the code block:

    (defun (if bool a b) (bool a b))
    (if true (*email-valentine* "I love you") (*email-valentine* "I don't love you"))

If we were to run this code strictly, we'd be in all kinds of trouble, and our valentine might rightly ask us why we didn't create `if` as a special form. However, with a lazy evaluation model, there's no need for conditional evaluation to be a privileged construct, and Valentine's day is saved.

## Representing Environments

We'll extend python dictionaries to support the `push` namespace operation required by `begin`.

In [67]:
class Environment:
    def all_bindings(self):
        d = self.parent_bindings.copy()
        for key in self.bindings:
            d[key] = self.bindings[key]
            
        return d
        
        return self.parent_bindings.union(self.bindings)
    
    def __init__(self, parent=None, bindings=None):
        self.parent = parent
        self.parent_bindings = {} if parent is None else parent.all_bindings()
        self.bindings = {} if bindings is None else bindings
        
    def __contains__(self, key):
        return key in self.bindings or key in self.parent_bindings
        
    def __getitem__(self, key):
        if key in self.bindings:
            return self.bindings[key]
        
        elif key in self.parent_bindings:
            return self.parent_bindings[key]
        
        else:
            raise KeyError(key)
            
    def __setitem__(self, key, value):
        self.bindings[key] = value
        return value
    
    def __repr__(self):
        return self.all_bindings().__repr__()
            
    def push(self, bindings=None):
        return Environment(parent=self, bindings=bindings)

# From Strings to Output

Now, we can compose the parser, the desugarer, and the evaluator to create a REPL for our language.

Since the REPL runs code line by line, we can eliminate the need for outer parentheses by automatically wrapping expressions run in the REPL in parentheses before parsing.

(In my opinion, `set x 2` is cleaner than `(set x 2)`, and curring also helps to eliminate needless parentheses. This has the additional benefit of representing the [SKI Calculus](https://en.wikipedia.org/wiki/SKI_combinator_calculus) with no modification -- `S K K` is a valid expression in the REPL if `S` and `K` are well-defined.)

In [68]:
def parse(stream, many_atoms = False):
    f = sepby(whitespace, atom) if many_atoms else atom
    
    ast, rest = f(stream)
    
    if ast is None:
        raise Exception("Parser failed")
    
    if rest:
        raise Exception("Parser failed to consume full input")
        
    return ast

In [69]:
def run_line(line, environment):
    return evaluate(desugar(parse(line)), environment)

def run_lines(lines, environment):
    return [evaluate(desugar(ast), environment) for ast in parse(lines, many_atoms=True)]

In [70]:
default_env = Environment()

In [71]:
def repl(library=None):
    env = default_env.push()
        
    if library is not None:
        run_lines(library, env)
    
    while True:
        code = input("λ ")
        if code == "quit" or code == "":
            break
                
        try:
            it = run_line(f"({code})", env)
            if it is not None:
                print(it)

        except Exception as e:
            raise(e)

# Library Writing

Good news! The rest of this section will be in lisp, so if you made it this far, congratulations!

Let's start by building a base library that encompasses most of the operations we've seen so far.

In [72]:
base_library = """
(defun (true x y) x)
(defun (false x y) y)
(defun (cons a b f) (f a b))
(defun (head l) (l true))
(defun (tail l) (l false))
(set ones (cons 1 ones))
(defun (if p a b) (p a b))
(defun (not x) (x false true))
(defun (flip x a b) (x b a))
(defun (bool_eq a b) (a b (not b)))
(defun (show_bool b) (b "true" "false"))
"""

Just for fun, here's the SKI calculus, which a very simple set of combinators capable of representing every possible mathematical operation.

In [73]:
ski_library = """
(defun (S x y z) (x z (y z)))
(defun (K x y) x)
(defun (I x) x)
"""

# Playing around in the REPL

Here's the finished product, have fun and show me what you make!

In [80]:
repl(base_library)

λ 1e12
1000000000000.0
λ quit


In [75]:
repl(ski_library)

λ 


# Let there be LISP

Let's attach some cooler bindings to it!

`Variable`, `Value`, and `Procedure` are the core "functional" abstractions of our language, while `Begin` and `SetEnv` are imperative since both rely on modifying the runtime environment. For now, we will focus on the functional core of the language, since this is easier to build on platforms that prioritize transforming data.



# Playing with Neural Nets

In [77]:
import numpy as np

In [78]:
def evaluate_neuron(weights, inputs):
    def activation(x):
        return -1 if x < 0 else 1
    
    return activation(sum(weight * inp for weight, inp in zip(weights, inputs)))

In [79]:
evaluate_neuron([1, 2], [-1, 0])

-1